In [2]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-08-19 02:12:58--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.98MB/s    in 0.2s    

2022-08-19 02:12:59 (4.98 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [7]:
# Start Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Pet_Products").getOrCreate()

# Load Dataset to Dataframe

In [8]:
# Load in user_data.csv from S3 into a DataFrame
from pyspark import SparkFiles

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), inferSchema=True, sep="\t")

df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [11]:
# Count the rows
df.count()

2643619

In [12]:
# Drop Null Values
df = df.dropna()

In [13]:
# Count the rows
df.count()

2643241

In [14]:
# Drop Duplicates
df = df.dropDuplicates()

In [15]:
# Count the rows
df.count()

2643241

In [19]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [20]:
# Change Datatype of review_date
df = df.withColumn("review_date",df.review_date.cast('date'))

In [21]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



#Transform Dataframe to Fit the Tables in the Schema Files

##Create review_id Table

In [22]:
# Create review_id Dataframe
review_id_df = df.select(["review_id","customer_id","product_id","product_parent","review_date"])
review_id_df.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100EO2AQP6B02|   33140435|B00BUQ4AV8|     805940477| 2014-10-12|
|R100IITU077KKZ|   25475634|B00RVY9DRO|     189955978| 2015-06-04|
|R100WIAVSCW4D1|   49229898|B004M8LM44|     419962202| 2013-07-31|
|R1023NXBAJ0BYT|   52227830|B0006IZH68|     470609741| 2004-12-23|
|R102CEVVIBXNFB|   11062585|B0014N70QO|     226028756| 2014-07-14|
|R102MO11LN79HV|   17155412|B0039LFVUG|     517238077| 2010-07-24|
|R102TPQOB6ECI4|   46107297|B008QVKBLS|     212972268| 2015-02-27|
|R1032IWOCCKRV1|   28307907|B00406YHRC|     178991753| 2014-01-08|
|R103QNPJX6OJ6D|   38188681|B003TUQWB4|     287424702| 2014-05-27|
|R104GDOC82SGT4|   49476810|B000YIWUXI|     178342731| 2015-03-13|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [23]:
# View Datatype for review_id_df
review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



##Create products Table

In [25]:
# Create products Dataframe
products_df = df.select(["product_id","product_title"])
products_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00BUQ4AV8|Evergreens Pet Do...|
|B00RVY9DRO|Volkman Avian Sci...|
|B004M8LM44|Dogtra YS300 Dog ...|
|B0006IZH68|thedog8it! Gourme...|
|B0014N70QO|Litter Kwitter To...|
|B0039LFVUG|So Bright LLC SO ...|
|B008QVKBLS|Super Bright 3W 2...|
|B00406YHRC|Prevue Pet Produc...|
|B003TUQWB4|Solvit Jumbo Stan...|
|B000YIWUXI|Safari® Cat Self-...|
+----------+--------------------+
only showing top 10 rows



In [26]:
# View Datatype for products 
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



##Create customers Table

In [27]:
# Create customers Dataframe
customers_df = df.groupBy("customer_id").count()
customers_df.show(10)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   37223012|    5|
|   52055901|    1|
|   16223525|    5|
|   28023775|    3|
|   51092659|    3|
|   44748307|   11|
|   33283869|    1|
|   14670648|   21|
|   21340218|    1|
|   40656071|    2|
+-----------+-----+
only showing top 10 rows



In [28]:
# Rename count Column
customers_df = customers_df.withColumnRenamed("count","customer_count")
customers_df.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   37223012|             5|
|   52055901|             1|
|   16223525|             5|
|   28023775|             3|
|   51092659|             3|
|   44748307|            11|
|   33283869|             1|
|   14670648|            21|
|   21340218|             1|
|   40656071|             2|
+-----------+--------------+
only showing top 10 rows



In [29]:
# View Datatype for customers 
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



##Create vine_table

In [30]:
# Create vine_table Dataframe
vine_table_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_table_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100EO2AQP6B02|          3|            2|          4|   N|
|R100IITU077KKZ|          5|            0|          0|   N|
|R100WIAVSCW4D1|          5|            3|          5|   N|
|R1023NXBAJ0BYT|          5|            0|          0|   N|
|R102CEVVIBXNFB|          1|            0|          2|   N|
|R102MO11LN79HV|          5|            0|          0|   N|
|R102TPQOB6ECI4|          3|            0|          0|   N|
|R1032IWOCCKRV1|          1|            2|          2|   N|
|R103QNPJX6OJ6D|          5|            1|          1|   N|
|R104GDOC82SGT4|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [31]:
# View Datatype for vine_table_df
vine_table_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



#Load Dataframe to RDS

In [ ]:
# Load Dataframe to RDS